In [ ]:
import pandas as pd

# Dataframe
df = pd.read_csv("sleep_health.csv")


id                     0
gender                 0
2014Age                0
2014BMI                0
2014SBP                0
                      ..
2014AlcoIntake         0
2015AlcoIntake         0
2014Alco_binary        0
2015Alco_binary        0
2014_2015AlcoIntake    0
Length: 61, dtype: int64
